In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F


In [12]:

device = torch.device('mps')
block_size = 3
batch_size = 4
max_iters = 2000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

In [13]:
with open('abc.txt', 'r', encoding='utf-8') as f: #wizard_of_oz.txt
    text = f.read()
chars = sorted(set(text))
print(len(chars))
vocab_size = len(chars)

35


In [14]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [15]:
n = int(len(data)* 0.8)

train_data = data[:n]
val_data = data[n:]
print(train_data)
print(val_data)

tensor([ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
        27, 28, 29, 30, 31, 32, 33, 34,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,  9, 10,
        11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
        29, 30, 31, 32, 33, 34,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,  9, 10, 11, 12,
        13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
        31, 32, 33, 34,  9, 10])
tensor([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
        29, 30, 31, 32, 33, 34,  0,  1,  2,  3,  4,  5,  6,  7,  8])


In [16]:

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[10, 11, 12],
        [30, 31, 32],
        [13, 14, 15],
        [27, 28, 29]], device='mps:0')
targets:
tensor([[11, 12, 13],
        [31, 32, 33],
        [14, 15, 16],
        [28, 29, 30]], device='mps:0')


In [17]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

1idyxv2hzixjq11km7qt6x6c2ycel1yxfyb4h4mgzqth5eeqfmcekjulfyk6q915999plr81zu9z5bfzuv17i46x6dtlluxmgw34mc7fqtlkicjitxr4f96zlu8j91txi5nvwyvwcnhzhc916fyxtdgyx137ir7bq5u2vs88ys1kctshbidk4fclzu8dc2hzx7y5dboj6qds8nu9tzaosz2w4f1irdyrl27rterhshp3cbhz9gfuihm9xjtzrd7bq11k6cn1wpdshluu9gw35fzuzxsd4hzuun6ctng68d73c95d7r5pf4moqhzk5lzcqu9s9qjtwbv42heirhu8swbfrgjtzu9bh5dpdeyxj55wy9ew3f4emvlupddc267bu9cns67bujum1113nhzsebzcfpd7rbhkvfvepy1zfx9gjp8r7xzcm1zy568blu99gpdumgqtzyx8xjpdlu8qtgwv9gidb4m34vbq9frc95luia7bx1tz2


In [18]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        print(f'Split is {split}')
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

Split is train
Split is val
step: 0, train loss: 4.409, val loss: 4.054
Split is train
Split is val
step: 250, train loss: 4.323, val loss: 3.986
Split is train
Split is val
step: 500, train loss: 4.203, val loss: 3.927
Split is train
Split is val
step: 750, train loss: 4.133, val loss: 3.887
Split is train
Split is val
step: 1000, train loss: 4.045, val loss: 3.822
Split is train
Split is val
step: 1250, train loss: 3.945, val loss: 3.727
Split is train
Split is val
step: 1500, train loss: 3.879, val loss: 3.687
Split is train
Split is val
step: 1750, train loss: 3.787, val loss: 3.639
3.7545089721679688


In [19]:
#context =  torch.zeros((1,1), dtype=torch.long, device=device)
context = torch.tensor((encode("1"),), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=50)[0].tolist())
print(generated_chars)

1irlofy5hmyxvtrnna3c3h14m373ybrgep53cmhqwyx944mv7bf
